In [141]:
class EstadoBloques:
    def __init__(self, p=[], m=['a','b'], l=['b','c'], s=[['a','c']]):
        
        self.pinza = p
        self.mesa = m
        self.libre = l
        self.sobre = s

    def __str__(self):
        r = 'Pinza: {}\nEn mesa: {}\nLibres: {}\nSobre: {}\n'.format(self.pinza,
                [mesa for mesa in self.mesa],
                [libre for libre in self.libre],
                [sobre for sobre in self.sobre])
        
        return r
    
    def __eq__(self,other):
        return self.__str__() == other.__str__()
    
    def __hash__(self):
        return hash(self.__str__())
    
    def is_final(self):
        return self.pinza == [] and self.mesa == ['c'] and ['c','b'] in self.sobre and ['b','a'] in self.sobre and self.libre == ['a']
    
    def coger1(self,x):
        
        r = False
        
        if self.pinza == [] and x in self.libre and x not in self.mesa:
            for s in self.sobre:
                if s[1] == x:
                    y,x = s
                    del self.libre[self.libre.index(x)]
                    del self.sobre[self.sobre.index(s)]
                    self.pinza.append(x)
                    self.libre.append(y)
                    r=True
                    
        return r
                    
                    
    def coger2(self,x):
        r = False
        if self.pinza == [] and x in self.libre and x in self.mesa:
            del self.libre[self.libre.index(x)]
            del self.mesa[self.mesa.index(x)]
            self.pinza.append(x)
            r = True
        return r
    
    def desapilar(self,x):
        r = False
        if x in self.pinza:
            del self.pinza[self.pinza.index(x)]
            self.mesa.append(x)
            self.libre.append(x)
            r = True
        return r
    
    def apilar(self,x,y):
        r=False
        if x in self.pinza and y in self.libre:
            del self.pinza[self.pinza.index(x)]
            del self.libre[self.libre.index(y)]
            self.libre.append(x)
            self.sobre.append([y,x])
            r=True
        return r


In [138]:
class Nodo:
    
    def __init__(self,estado,padre):
        self.estado = estado
        self.padre = padre
        
    # el estado es un estado
    def getEstado(self):
        return self.estado
    
    # el padre es un nodo
    def getPadre(self):
        return self.padre
    
    def getCamino(self):
        camino = []
        p = self
        
        while p:
            camino.append(p.getEstado())
            p = p.getPadre()
            
        return list(reversed(camino))
    
    def __str__(self):
        return self.estado.__str__()# + 'Padre: ' + self.getPadre().estado.__str__()
        

In [142]:
cajas = ['a','b','c']

def sucesores(nodo):
    # estadoViejo = EstadoBloques(estado.pinza, estado.mesa, estado.libre, estado.sobre) 
    
    estado = nodo.getEstado()
    
    pinzacopy = estado.pinza[:]
    mesacopy = estado.mesa[:]
    librecopy = estado.libre[:]
    sobrecopy = estado.sobre[:]
    
    estadoModificar = EstadoBloques(pinzacopy, mesacopy, librecopy, sobrecopy)
    
    listaSucc = []
    
    if pinzacopy:
        p = pinzacopy[0]
        for x in librecopy:
            boo = estadoModificar.apilar(p,x)
            if boo:
                listaSucc.append(Nodo(estadoModificar,nodo))
                estadoModificar = EstadoBloques(pinzacopy, mesacopy, librecopy, sobrecopy)
        estadoModificar.desapilar(p)
        listaSucc.append(Nodo(estadoModificar,nodo))
        estadoModificar = EstadoBloques(pinzacopy, mesacopy, librecopy, sobrecopy)
    else:
        for x in librecopy:
            if x in mesacopy:
                estadoModificar.coger2(x)
                listaSucc.append(Nodo(estadoModificar,nodo))
                estadoModificar = EstadoBloques(pinzacopy, mesacopy, librecopy, sobrecopy)
            else:
                estadoModificar.coger1(x)
                listaSucc.append(Nodo(estadoModificar,nodo))
                estadoModificar = EstadoBloques(pinzacopy, mesacopy, librecopy, sobrecopy)
                
    return listaSucc
    

In [126]:
def primero_profundidad(inicial):
    
    while True:
        if inicial==[]:
            return None

        actual = inicial[0]

        if actual.getEstado().is_final():
            return inicial

        succ=sucesores(actual)
        inicial = succ+inicial[1:]

    # succ = sucesores(nodos[0])

    # nodos = succ + nodos[1:]
    
    

In [62]:
prueba = EstadoBloques()
print(prueba)

Pinza: []
En mesa: ['a', 'b']
Libres: ['b', 'c']
Sobre: [['a', 'c']]



In [63]:
prueba.apilar('a','b')

False

In [64]:
prueba.coger1('c')

True

In [65]:
print(prueba.sobre)

[]


In [66]:
print(prueba)

Pinza: ['c']
En mesa: ['a', 'b']
Libres: ['b', 'a']
Sobre: []



In [67]:
prueba.desapilar('c')

True

In [68]:
print(prueba)

Pinza: []
En mesa: ['a', 'b', 'c']
Libres: ['b', 'a', 'c']
Sobre: []



In [69]:
prueba.coger2('b')

True

In [70]:
prueba.apilar('b','c')

True

In [71]:
print(prueba)

Pinza: []
En mesa: ['a', 'c']
Libres: ['a', 'b']
Sobre: [['c', 'b']]



In [72]:
prueba.is_final()

False

In [73]:
prueba.coger2('a')
prueba.apilar('a','b')

True

In [74]:
print(prueba)

Pinza: []
En mesa: ['c']
Libres: ['a']
Sobre: [['c', 'b'], ['b', 'a']]



In [75]:
prueba.is_final()

True

In [76]:
pru2 = EstadoBloques()

In [81]:
print(pru2)

Pinza: []
En mesa: ['c']
Libres: ['a']
Sobre: [['c', 'b'], ['b', 'a']]



In [143]:
nodo = Nodo(pru2,None)

lista = sucesores(nodo)

In [144]:
for j in lista:
    print(j)

Pinza: ['a']
En mesa: ['c']
Libres: ['b']
Sobre: [['c', 'b']]



In [80]:
sucesores(nodo)

In [85]:
prueba is pru2

False

In [86]:
prueba == pru2

True

In [93]:
pru3 = EstadoBloques([], ['a','b'], ['b','c'], [['a','c']])

In [94]:
prueba == pru3

False

In [145]:
print(pru3)

Pinza: []
En mesa: ['a', 'b']
Libres: ['b', 'c']
Sobre: [['a', 'c']]



In [147]:
for k in sucesores(Nodo(pru3,None)):
    print(k)
    
len(sucesores(Nodo(pru3,None)))

Pinza: ['b']
En mesa: ['a']
Libres: ['c']
Sobre: [['a', 'c']]



1

In [148]:
pru3.coger1('c')

True

In [149]:
print(pru3)

Pinza: ['c']
En mesa: ['a', 'b']
Libres: ['b', 'a']
Sobre: []



In [103]:
nodoInicial = Nodo(pru3,None)
nodoFinal = Nodo(prueba,None)

In [136]:
lista_algoritmo = primero_profundidad([nodoInicial])

KeyboardInterrupt: 

In [ ]:
for i in lista_algoritmo:
    print(i)